In [1]:
"""
[1] obje-kun プロセスフローからオブジェクトリストを出すやつ
入力：自然言語のプロセスフロー + 制約条件
e.g. 新しいチューブに10 mLの培地と1 mLのサプリメントを入れる … + この機械は50 mlチューブを使えない
出力：[{“オブジェクト名”: “xx”, “個数”: “xx”, “制約”: “???”}, …]
e.g. [{“オブジェクト名”: “10 mlチューブ”, “個数”: 2, “制約”: “0 ml以上10 mlまで”}, …]
オブジェクトリストでプロセスフローが実行可能かをどう判定する？
言語モデルでつくるとこれが難しそう
使えるものの種類と在庫を先に与える？
であればルールベースでいけそう
"""

import config as settings
from typing import List, Dict
from dataclasses import dataclass
from prompts import PROMPT
import openai
import json
from datetime import datetime
import os

@dataclass
class Object:
    name: str # e.g. 1.5 ml tube
    quantity: int # e.g. 1
    init_content: str # e.g. DMEM

@dataclass
class Constraints:
    allowed_objects: List[Object]

def init_settings():
    openai.api_key = settings.OPENAI_API_KEY

    # model_list = openai.Model.list()
    # print(f'OpenAI Model List: ', model_list)
    return

def _extract_results_from_gpt_response(result: str) -> List[Object]:
    """
    Extract JSON results from GPT response.
    """
    # pre process
    # delete ```json``` if any
    result = result.replace('```json', '').replace('```', '')

    print(f'result: {result}')

    text_to_dict = lambda x: json.loads(x.replace('\'', '\"'))
    results = text_to_dict(result.replace('extracted_results:', '').replace('extracted_results_end:', '').strip())
    objects = [Object(name=object['labware_name'], quantity=object['labware_quantity'], init_content=object['init_content']) for object in results]
    return objects

def check_constraints_and_generate_error_messages(object_list: List[Object], constraints: Constraints, remove=False) -> List[str]:
    """
    Given the extracted object list and constraints, check if the object list satisfies the constraints.
    If not, generate error messages and return them.
    """
    allowed_object_names = [object.name for object in constraints.allowed_objects]
    allowed_objects_original_quantity = {object.name: object.quantity for object in constraints.allowed_objects}
    allowed_objects_quantity = {object.name: object.quantity for object in constraints.allowed_objects}
    err_messages = []
    for object in object_list:
        if object.name not in allowed_object_names:
            if remove:
                object_list.remove(object)
                err_msg = f'Removed {object.name} from object list because it is not allowed. Please use different object instead.'
            else:
                err_msg = f'You have used {object.name} that is not allowed. Please use different object instead.'
            err_messages.append(err_msg)

        if object.name not in allowed_objects_quantity:
            err_msg = f'You have used {object.name} that is not allowed. Please use different object instead.'
            err_messages.append(err_msg)
            continue

        allowed_objects_quantity[object.name] -= object.quantity
        if allowed_objects_quantity[object.name] < 0:
            err_msg = f'You have used {object.name} more than allowed quantity. Please use different object instead.'
            err_messages.append(err_msg)

    return err_messages

def get_process_flow_text(model='gpt-3.5-turbo-16k') -> str:
    # ask GPT
    prompt = """
Could you generate a process flow text of biological experiments? For example,

Example 1:

Process flow text:
```
1) Add 100 uL of sample from 1.5 ml tube to 96 well plate.
2) Transfer 10 ml medium to 15 ml tube.
3) Add 300 µl of the medium from 15 ml tube to 96 well plate.
```

Example 2:

Process flow text:
```
Add 5 ml DMEM from 15 ml tube to 90 mm dish.
And then add 5 ml MEMa from 15 ml tube to the same dish.
Finally, add 10 µl sample from 1.5 ml tube to the same dish.
```

Please make sure you only generate process flow text without any other text so that we can extract the results correctly.

Now it's your turn to fill the process flow text.

Process flow text:
    """
    res = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who is an expert in biology, computer science, and engineering."},
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    should_be_process_flow: str = res['choices'][0]['message']['content']
    return should_be_process_flow

def get_object_list(process_flow: str, constraints: Constraints, model='gpt-3.5-turbo-16k'):
    """
    process_flow: str
    constraints: dict
    """
    prompt = PROMPT['v1'].replace('{__process_flow_text__}', process_flow)
    # ask GPT
    res = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who is an expert in biology, computer science, and engineering."},
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    print(f'total token: ', res['usage'])
    answer = res['choices'][0]['message']['content']
    results = _extract_results_from_gpt_response(answer)
    errors = check_constraints_and_generate_error_messages(results, constraints)

    save_result(prompt, constraints, results, errors, model=model)
    return results, errors

def save_result(
    prompt: str,
    constraints: Constraints,
    result: str,
    errors: str,
    version: str = 'v1',
    model: str = 'gpt-3.5-turbo-16k'
):
    current_datetime = datetime.now().strftime('%Y%m%d%H%M%S')
    filename = f'result_{version}_{model}_{current_datetime}.txt'
    file_path = os.path.join(settings.RESULT_SAVE_PATH, filename)
    with open(file_path, 'w') as f:
        f.write(f'*************PROMPT*************\n{prompt}\n\n*************CONSTRAINTS*************\n{str(constraints)}\n\n*************RESULT*************\n{result}\n\n*************ERRORS*************\n{errors}')



def run_example():
    example_process_flow = """
    1) Add 100 uL of sample from 1.5 ml tube to 96 well plate.
    2) Transfer 10 ml medium to 15 ml tube.
    3) Add 300 µl of the medium from 15 ml tube to 96 well plate."""
    example_constraints = Constraints(allowed_objects=[
        Object(name='1.5 ml tube', quantity=1, init_content='sample'),
        Object(name='96 well plate', quantity=0, init_content=''),
        Object(name='15 ml tube', quantity=1, init_content='medium')
    ])
    res, errors = get_object_list(example_process_flow, example_constraints)
    print(f'\n\nProcess Flow ******************\n{example_process_flow}\n\nConstraints ******************\n{example_constraints}\n\nExtracted Object List******************\n{res}\n\nError messages******************\n{errors}')
    return res, errors

def run(process_flow: str, constraints: Constraints):
    res, errors = get_object_list(process_flow, constraints)
    print(f'\n\nProcess Flow ******************\n{process_flow}\n\nConstraints ******************\n{constraints}\n\nExtracted Object List******************\n{res}\n\nError messages******************\n{errors}')
    return res, errors

init_settings()
res, errors = run_example()

total token:  {
  "prompt_tokens": 960,
  "completion_tokens": 80,
  "total_tokens": 1040
}
result: [
    {"labware_name": "1.5 ml tube", "labware_quantity": 1, "init_content": "sample"},
    {"labware_name": "96 well plate", "labware_quantity": 1, "init_content": "medium"},
    {"labware_name": "15 ml tube", "labware_quantity": 1, "init_content": ""}
]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/takashimac/Library/CloudStorage/Dropbox/理研_神田先生/FormationFormatter/Obje-chan/results/result_v1_gpt-3.5-turbo-16k_20231002164439.txt'

In [5]:
process_flow_constraints_pair = [
        {
        'process_flow': """
1) Add 100 uL of sample from 1.5 ml tube to 96 well plate.
2) Transfer 10 ml medium to 15 ml tube.
3) Add 300 µl of the medium from 15 ml tube to 96 well plate.
        """,
        'constraints': Constraints(
                allowed_objects=[
                        Object(name='1.5 ml tube', quantity=1, init_content='sample'),
                        Object(name='96 well plate', quantity=1, init_content=''),
                        Object(name='15 ml tube', quantity=1, init_content='medium')]
                )
        },
        {
        'process_flow': get_process_flow_text(),
        'constraints': Constraints(
                allowed_objects=[
                        Object(name='1.5 ml tube', quantity=1, init_content='sample'),
                        Object(name='96 well plate', quantity=1, init_content=''),
                        Object(name='15 ml tube', quantity=1, init_content='medium')]
                )
        },
]

idx = 1
res, errors = run(
        process_flow=process_flow_constraints_pair[idx]['process_flow'],
        constraints=process_flow_constraints_pair[idx]['constraints']
        )

total token:  {
  "prompt_tokens": 1114,
  "completion_tokens": 269,
  "total_tokens": 1383
}
result: [
    {"labware_name": "1.5 ml tube", "labware_quantity": 1, "init_content": "sample"},
    {"labware_name": "96 well plate", "labware_quantity": 1, "init_content": "medium"},
    {"labware_name": "15 ml tube", "labware_quantity": 1, "init_content": ""}
]

[
    {"labware_name": "15 ml tube", "labware_quantity": 2, "init_content": "DMEM"},
    {"labware_name": "90 mm dish", "labware_quantity": 1, "init_content": ""},
    {"labware_name": "1.5 ml tube", "labware_quantity": 1, "init_content": "sample"}
]

[
    {"labware_name": "1.5 ml tube", "labware_quantity": 1, "init_content": "sample"},
    {"labware_name": "96 well plate", "labware_quantity": 1, "init_content": ""},
    {"labware_name": "15 ml tube", "labware_quantity": 3, "init_content": "trypsin"},
    {"labware_name": "15 ml tube", "labware_quantity": 2, "init_content": "PBS"}
]


JSONDecodeError: Extra data: line 7 column 1 (char 257)

In [3]:
def call_n_times(n=100):
    import time
    idx = 1
    for i in range(n):
        print(f'\n\n****************** Iteration {i}/{n} ******************\n\n')
        try:
            res, errors = run(
                process_flow=process_flow_constraints_pair[idx]['process_flow'],
                constraints=process_flow_constraints_pair[idx]['constraints']
                )
        except Exception as e:
            print(f'Error in iteration {i}')
            import traceback
            traceback.print_exc()
        time.sleep(60)

call_n_times(n=1)



****************** Iteration 0/1 ******************


total token:  {
  "prompt_tokens": 1036,
  "completion_tokens": 109,
  "total_tokens": 1145
}
result: 
[
    {"labware_name": "1.5 ml tube", "labware_quantity": 2, "init_content": "bacterial culture"},
    {"labware_name": "15 ml tube", "labware_quantity": 2, "init_content": ""},
    {"labware_name": "96 well plate", "labware_quantity": 1, "init_content": ""},
    {"labware_name": "spectrophotometer", "labware_quantity": 1, "init_content": ""}
]



Process Flow ******************
Add 1 ml of bacterial culture from a 1.5 ml tube to a 15 ml tube. 

Then, transfer 200 µl of the culture from the 15 ml tube to a 96 well plate. 

After that, add 10 µg of DNA from a 1.5 ml tube to the 15 ml tube.

Next, incubate the 96 well plate at 37°C for 1 hour.

Following the incubation, add 100 µl of the medium from the 15 ml tube to the 96 well plate.

Finally, measure the absorbance at 600 nm using a spectrophotometer to analyze the bacterial gro

In [2]:
openai.Model.list()

<OpenAIObject list at 0x1077a5590> JSON: {
  "object": "list",
  "data": [
    {
      "id": "text-search-babbage-doc-001",
      "object": "model",
      "created": 1651172509,
      "owned_by": "openai-dev",
      "permission": [
        {
          "id": "modelperm-s9n5HnzbtVn7kNc5TIZWiCFS",
          "object": "model_permission",
          "created": 1695933794,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": true,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "text-search-babbage-doc-001",
      "parent": null
    },
    {
      "id": "curie-search-query",
      "object": "model",
      "created": 1651172509,
      "owned_by": "openai-dev",
      "permission": [
        {
          "id": "modelperm-8aqdyZaKtD3MD831mGbqh1MD",
          "object": "mod